<a href="https://colab.research.google.com/github/marcory-hub/hailo/blob/main/DFC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# PAS INSTALL EN .pt naar onnx ALS ULTRALYTICS WEER VEILIG IS


In [ ]:
!pip install onnx
!pip install onnxruntime
!pip install torch

In [ ]:
import onnx
import onnxruntime as ort
import torch

# Load the ONNX model
onnx_model = onnx.load("/content/gdrive/MyDrive/best9.onnx")
onnx.checker.check_model(onnx_model)
print("ONNX model is valid!")


# Test the ONNX model with ONNX Runtime
dummy_input = torch.randn(1, 3, 640, 640).numpy()
ort_session = ort.InferenceSession("/content/gdrive/MyDrive/best9.onnx")

# Run the model using the correct input name
outputs = ort_session.run(None, {"images": dummy_input})
print(outputs[0])


In [ ]:
#!sudo apt-get update # run if package is not found

# Install dependencies as described in system requirements:
# https://hailo.ai/developer-zone/documentation/dataflow-compiler-v3-29-0/?sp_referrer=install/install.html
!sudo apt-get install -y python3-dev python3-distutils python3-tk libfuse2 graphviz libgraphviz-dev

In [ ]:
# Make venv
!pip install --upgrade pip virtualenv
!virtualenv venv

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Download hailo dataflow compiler (python 3.10): https://hailo.ai/developer-zone/software-downloads/

Copy it to googledrive

In [ ]:
#Install DFC (adjust the path to the .whl if needed)
!venv/bin/pip install  /content/gdrive/MyDrive/hailo_dataflow_compiler-3.29.0-py3-none-linux_x86_64.whl

In [13]:
!venv/bin/hailo --version

[info] Current Time: 20:27:26, 12/08/24
[info] CPU: Architecture: x86_64, Model: AMD EPYC 7B12, Number Of Cores: 2, Utilization: 59.3%
[info] Memory: Total: 12GB, Available: 10GB
[info] System info: OS: Linux, Kernel: 6.1.85+
[info] Hailo DFC Version: 3.29.0
[info] HailoRT Version: Not Installed
[info] PCIe: No Hailo PCIe device was found
[info] Running `hailo --version`
Hailo Dataflow Compiler v3.29.0


In [14]:
from hailo_sdk_client import ClientRunner

# Define the ONNX model path and configuration
onnx_path = "/content/gdrive/MyDrive/best9.onnx"
onnx_model_name = "best9"
chosen_hw_arch = "hailo8"  # Specify the target hardware architecture

# Initialize the ClientRunner
runner = ClientRunner(hw_arch=chosen_hw_arch)

# Use the recommended end node names for translation
end_node_names = [
    "/model.14/cv2.0/cv2.0.2/Conv",  # P4 regression_layer
    "/model.14/cv3.0/cv3.0.2/Conv",  # P4 cls_layer
    "/model.14/cv2.1/cv2.1.2/Conv",  # P5 regression_layer
    "/model.14/cv3.1/cv3.1.2/Conv",  # P5 cls_layer,
]

try:
    # Translate the ONNX model to Hailo's format
    hn, npz = runner.translate_onnx_model(
        onnx_path,
        onnx_model_name,
        end_node_names=end_node_names,
        net_input_shapes={"input": [16, 3, 640, 640]},  # Adjust input shapes if needed
    )
    print("Model translation successful.")
except Exception as e:
    print(f"Error during model translation: {e}")
    raise

# Save the Hailo model HAR file
hailo_model_har_name = f"{onnx_model_name}_hailo_model.har"
try:
    runner.save_har(hailo_model_har_name)
    print(f"HAR file saved as: {hailo_model_har_name}")
except Exception as e:
    print(f"Error saving HAR file: {e}")

ModuleNotFoundError: No module named 'hailo_sdk_client'